In [9]:
import rasterio
from rasterio.warp import transform
from shapely.geometry import Polygon

def get_image_corners(image_path):
    with rasterio.open(image_path) as src:
        
        bounds = src.bounds
    
        crs = src.crs
        
        
        corners = [
            (bounds.left, bounds.top),     
            (bounds.right, bounds.top),    
            (bounds.right, bounds.bottom),
            (bounds.left, bounds.bottom)   
        ]
        
       
        corners_wgs84 = transform(crs, {'init': 'epsg:4326'}, [corner[0] for corner in corners], [corner[1] for corner in corners])
        
       
        corners_wgs84 = list(zip(corners_wgs84[0], corners_wgs84[1]))
        
    return corners_wgs84

def calculate_polygon_area(polygon_coords):
    polygon = Polygon(polygon_coords)
    area = polygon.area  
    
  
    area_sqm = area * (111320 * 111320)
    
    return area_sqm

image_path = 'images.tif'
corners_wgs84 = get_image_corners(image_path)
print("Corners (lat, lon):", corners_wgs84)


image_area_sqm = calculate_polygon_area(corners_wgs84)
print(f"Image area: {image_area_sqm:.2f} square meters")

Corners (lat, lon): [(-43.6116600211, -22.8948639367), (-43.6096921247, -22.8948639367), (-43.6096921247, -22.8966871589), (-43.6116600211, -22.8966871589)]
Image area: 44461.92 square meters


In [19]:
import rasterio
import numpy as np
from shapely.geometry import Polygon, shape
import json
import cv2

def get_image_area_in_pixels(image_path):
    with rasterio.open(image_path) as src:
        width = src.width
        height = src.height
        image_area_pixels = width * height
    return image_area_pixels, width, height

def get_image_corners(image_path):
    with rasterio.open(image_path) as src:
        bounds = src.bounds
        crs = src.crs
        
        corners = [
            (bounds.left, bounds.top),
            (bounds.right, bounds.top),
            (bounds.right, bounds.bottom),
            (bounds.left, bounds.bottom)
        ]
        
        corners_wgs84 = transform(crs, {'init': 'epsg:4326'}, [corner[0] for corner in corners], [corner[1] for corner in corners])
        corners_wgs84 = list(zip(corners_wgs84[0], corners_wgs84[1]))
        
    return corners_wgs84

def calculate_image_area(image_path):
    corners_wgs84 = get_image_corners(image_path)
    polygon = Polygon(corners_wgs84)
    area = polygon.area
    
    area_sqm = area * (111320 * 111320)
    return area_sqm

def pixel_to_meter_conversion_factor(image_path):
    image_area_pixels, width, height = get_image_area_in_pixels(image_path)
    image_area_sqm = calculate_image_area(image_path)
    conversion_factor = image_area_sqm / image_area_pixels
    return conversion_factor, width, height


In [32]:
def calculate_building_areas_from_masks(mask_path, conversion_factor):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    num_labels, labels_im = cv2.connectedComponents(mask)
    
    building_areas_pixels = []
    for label in range(0, num_labels):
        area_pixels = np.sum(labels_im == label)
        print(area_pixels)
        building_areas_pixels.append(area_pixels)
    
    building_areas_sqm = [area_pixels * conversion_factor for area_pixels in building_areas_pixels]
    return building_areas_sqm


In [6]:
# def get_building_areas_from_geojson(geojson_path):
#     with open(geojson_path, 'r') as file:
#         geojson_data = json.load(file)
    
#     building_areas_geojson = []
#     for feature in geojson_data['features']:
#         polygon = shape(feature['geometry'])
#         area = polygon.area * (111320 * 111320)
#         building_areas_geojson.append(area)
    
#     return building_areas_geojson



In [33]:
def main(image_path, mask_path):
    conversion_factor, width, height = pixel_to_meter_conversion_factor(image_path)
    # print(conversion_factor,width,height)
    building_areas_sqm = calculate_building_areas_from_masks(mask_path, conversion_factor)
    # building_areas_geojson = get_building_areas_from_geojson(geojson_path)
    
    print(f"Building areas (square meters) from binary masks: {building_areas_sqm}")
    # print(f"Building areas (square meters) from GeoJSON: {building_areas_geojson}")

# Example usage
image_path = 'images.tif'
mask_path = 'predicted_mask.tif'
# geojson_path = "C:\\Users\\Lenovo\\Desktop\\model\\dataset\\labels\\00e7b749-6c4f-4b11-b25e-612970f62f60.geojson"
main(image_path, mask_path)

65536
Building areas (square meters) from binary masks: [16385.813722290935]
